# Run Command Job

This notebook demonstrates how to submit the `train_model.py` script as a Command Job using the Azure ML SDK v2.

In [ ]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

# Workspace details
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<WORKSPACE_NAME>"

# Connect to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
)

## Define the Command Job

We use the `command` function to define the job details.

*   `code`: The source directory containing the script.
*   `command`: The command to run. Note the use of `${{inputs.data}}` to reference the input data path.
*   `environment`: The environment to run the script in (here using a curated Azure ML environment).
*   `compute`: The compute target name.
*   `inputs`: The input dictionary mapping input names to data assets.

In [ ]:
# Configure the command job
job = command(
    code="./src",
    command="python train_model.py --data ${{inputs.diabetes}} --learning_rate ${{inputs.learning_rate}}",
    inputs={
        "diabetes": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
        ),
        "learning_rate": 0.1
    },
    environment="azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    compute="cpu-cluster",
    display_name="diabetes-training-job",
    experiment_name="diabetes-experiment",
    description="Train a regression model on diabetes data"
)

In [ ]:
# Submit the job
returned_job = ml_client.jobs.create_or_update(job)
print(f"Job created with URI: {returned_job.studio_url}")